In [1]:
# Importing requird python packages
# Path to hsmpy package
HSMPY_PATH = r'C:\Users\MR068144\Downloads\Python Scripts'
import sys, os, arcpy, csv, json, math
sys.path.append(HSMPY_PATH)
import hsmpy3
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
from IPython.display import Image
from IPython.core.display import HTML 
import subprocess 
import webbrowser
import time
import copy
import shutil
from time import gmtime, strftime
from ipywidgets import FloatProgress
print(strftime("%Y-%m-%d %H:%M:%S"))

2018-09-07 09:51:08


In [2]:
# Defining Parameteres
print(strftime("%Y-%m-%d %H:%M:%S"))
Years = range(2012,2017)
MaxOpenProcesses = 15
Project_DIR  = r'\\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO00_OtherStates\NewMexico\3_WorkData\7_NetworkScreening'
ReceivedDir  = Project_DIR + '\\2.Received_Data'
Project_GDB  = Project_DIR + '\\5.NM_NS_GDB.gdb'

Src_Loc  = ReceivedDir + '\\crash_asof_11jun2018.csv'
Src_Unit = ReceivedDir + '\\vehicle_asof_11jun2018.csv'
Src_Occ  = ReceivedDir + '\\occupant_asof_11jun2018.csv'

HPMS105 = {year:ReceivedDir + '\\v105\\jg_request.gdb\\HPMS{}Data'.format(year) for year in Years}
HPMS10  = {year:ReceivedDir + '\\v10\\jg_request.gdb\\HPMS{}Data'.format(year) for year in Years}

Routes  = {year:Project_GDB + '\\Routes_{}'.format(year) for year in Years}
AttTabs = {year:Project_GDB + '\\SegAtt_{}'.format(year) for year in Years}
Segs    = {year:Project_GDB + '\\Seg_{}'.format(year) for year in Years}
CrashData = {year:Project_GDB + '\\Crash_{}'.format(year) for year in range(2012,2018)}
Segs[2017] = Segs[2016]


UTM13N = arcpy.SpatialReference(26913)
NAD83_NM = arcpy.SpatialReference(6529)

RouteID = "ROUTE_ID"
BEG_STA = 'BEG_POINT'
END_STA = 'END_POINT'
Fields = ['F_SYSTEM','THROUGH_LANES','FACILITY_TYPE','AADT','MEDIAN_TYPE','SPEED_LIMIT','LANE_WIDTH']

BridgeUnder = ReceivedDir + '\\v10\\jg_request.gdb\\BridgeUnder'
BridgeLRS   = ReceivedDir + '\\v10\\jg_request.gdb\\BridgeLRS'
Signals     = ReceivedDir + '\\v10\\jg_request.gdb\\SignalizedIntersections'

2018-09-07 09:51:08


In [3]:
# Add Roadway Facility Type
print(strftime("%Y-%m-%d %H:%M:%S"))
def FindFType(RID,F_SYSTEM,FACILITY_TYPE,THROUGH_LANES,URBAN_CODE,MEDIAN_TYPE):
    if FACILITY_TYPE==2 and THROUGH_LANES>0:
        S1 = 'U'
        if URBAN_CODE in [99999,0]:
            S1 = 'R'

        S2 = str(int(THROUGH_LANES))
        if THROUGH_LANES in [3,5,7,9] and F_SYSTEM == 1:
            S2 = str(int(THROUGH_LANES-1))

        S3 = 'U'
        if MEDIAN_TYPE in [2,3,4,5,6,7]:
            S3 = 'D'
        if F_SYSTEM in [1,2]:
            S3 = 'F'
        if F_SYSTEM == 7:
            S3 = 'L'
        return (S1+S2+S3)
    else:
        if 'X' in RID:
            return('Ramp')
        return('')

for year in Years:
    if not 'FType' in [f.name for f in arcpy.ListFields(Segs[year])]:
        arcpy.management.AddField(Segs[year],'FType','Text')
    uc = arcpy.UpdateCursor(Segs[year])
    f = FloatProgress(min=0, max=int(str(arcpy.GetCount_management(Segs[year]))),description = str(year))
    display(f)
    for row in uc:
        ft = FindFType(RID           = row.getValue('ROUTE_ID'),
                       FACILITY_TYPE = row.getValue('FACILITY_TYPE'),
                       F_SYSTEM      = row.getValue('F_SYSTEM'),
                       MEDIAN_TYPE   = row.getValue('MEDIAN_TYPE'),
                       THROUGH_LANES = row.getValue('THROUGH_LANES'),
                       URBAN_CODE    = row.getValue('URBAN_CODE'))
        if ft!='':
            row.setValue('FType',ft)
            uc.updateRow(row)
        f.value += 1
    del row
    del uc
    try:
        arcpy.management.Delete(AttTabs[int(year)])
    except:pass
    arcpy.management.CopyRows(in_rows=Segs[int(year)],out_table=AttTabs[int(year)])    
print(strftime("%Y-%m-%d %H:%M:%S"))

2018-08-06 10:36:34


FloatProgress(value=0.0, description='2012', max=9116.0)

FloatProgress(value=0.0, description='2013', max=7217.0)

FloatProgress(value=0.0, description='2014', max=9027.0)

FloatProgress(value=0.0, description='2015', max=10566.0)

FloatProgress(value=0.0, description='2016', max=224743.0)

2018-08-06 10:45:03


In [79]:
# reading Segment Info
print(strftime("%Y-%m-%d %H:%M:%S"))
Segs_Dict = {year:hsmpy3.common.FCtoDF(Segs[year],True) for year in Years}
Segs_DF = pd.DataFrame()
for year in Years:
    df = Segs_Dict[year]
    df['P3'] = [r.ROUTE_ID.split('-')[2] for i,r in df.iterrows()]
    #df = df[(df.P3 =='P')]
    df['Mileage'] = [r/5280.0 for r in list(df.Shape_Length)]
    df['Year'] = year
    #df['AADT'] = [ConvToFloat(aadt) for aadt in list(df.AADT)]
    df['VMT'] = [aadt*365*leng for aadt,leng in zip(list(df.AADT),list(df.Mileage))]
    Segs_DF = pd.concat([Segs_DF,df])
    print(year)
Segs_DF = Segs_DF.fillna(0)

2018-08-07 08:22:20
2012
2013
2014
2015
2016


In [86]:
FTypes = ['R4F','U4F','U6F','R2U','U2U','R4U','R4D','U4U','U4D','U6U']
FTypeDesc = {
    'R4F':'Rural 4 Lane Freeways',
    'U4F':'Urban 4 Lane Freeways',
    'U6F':'Urban 6 Lane Freeways',
    'R2U':'Rural 2 Lane Undivided Roads',
    'U2U':'Urban 2 Lane Undivided Roads',
    'R4U':'Rural 4 Lane Undivided Roads',
    'R4D':'Rural 4 Lane Divided Roads',
    'U4U':'Urban 4 Lane Undivided Roads',
    'U4D':'Urban 4 Lane Divided Roads',
    'U6U':'Urban 6 Lane Undivided Roads',
}
Segs_DF_Sel = Segs_DF[(Segs_DF.FType.isin(FTypes)) & (Segs_DF.AADT>0)]
df = pd.DataFrame(Segs_DF_Sel.groupby(['FType'])['Mileage','VMT','K','A','B','PedK','PedA','PedB'].aggregate(np.sum))
df.VMT = df.VMT/100000000.0
df = df.loc[FTypes]
df['Peer Group Description'] = [FTypeDesc[f] for f in FTypes]
df = df[['Peer Group Description','Mileage','VMT','K','A','B','PedK','PedA','PedB']]
df.loc['Total'] = ['',sum(df.Mileage),sum(df.VMT),sum(df.K),sum(df.A),sum(df.B),sum(df.PedK),sum(df.PedA),sum(df.PedB)]
df[['K', 'A','B','PedK','PedA','PedB']] = df[['K', 'A','B','PedK','PedA','PedB']].astype(int)
df.index.name = None
df.columns.name='Peer Group Codes'
pd.options.display.float_format = '{:,.2f}'.format
df[['Peer Group Description','Mileage','VMT','K','A','B']]

Peer Group Codes,Peer Group Description,Mileage,VMT,K,A,B
R4F,Rural 4 Lane Freeways,"2,133.73",103.86,118,131,461
U4F,Urban 4 Lane Freeways,271.47,23.21,33,63,209
U6F,Urban 6 Lane Freeways,90.76,26.33,6,45,133
R2U,Rural 2 Lane Undivided Roads,"23,734.30",128.70,101,153,515
U2U,Urban 2 Lane Undivided Roads,"3,142.39",51.60,9,36,96
R4U,Rural 4 Lane Undivided Roads,"2,991.40",54.81,35,67,150
R4D,Rural 4 Lane Divided Roads,751.35,17.70,16,20,74
U4U,Urban 4 Lane Undivided Roads,"1,385.44",69.30,24,33,120
U4D,Urban 4 Lane Divided Roads,240.13,11.47,4,17,25
U6U,Urban 6 Lane Undivided Roads,197.38,21.97,1,8,29


In [201]:
df = pd.DataFrame(Segs_DF_Sel.groupby(['FType','F_SYSTEM'])['Mileage','VMT','K','A','B','PedK','PedA','PedB'].aggregate(np.sum))
df.VMT = df.VMT/100000000.0
df = df.fillna(0)
df[['K', 'A','B','PedK','PedA','PedB']] = df[['K', 'A','B','PedK','PedA','PedB']].astype(int)
df

Mileage    VMT    K    A     B  PedK  PedA  PedB
FType F_SYSTEM                                                  
R2U   0           14.91   0.19    0    0     3     0     0     0
      3        1,888.10  17.18   32   38   150     3     0     1
      4        4,814.33  31.99   41   75   262     6     3     3
      5        9,772.75  31.76   80  124   406     7     2     8
      6        1,938.55   4.79    6   20    80     0     2     4
R4D   0            1.31   0.05    0    0     1     0     0     0
      3          711.04  16.94   18   34   120     3     6     0
      4           29.17   0.32    1    0     2     0     0     0
      5            8.30   0.36    0    0     2     0     0     0
      6            1.53   0.03    0    1     3     0     0     1
R4F   1        2,133.73 103.87  119  131   464    19     2     3
R4U   0            0.35   0.02    0    0     0     0     0     0
      3        1,695.70  38.97   53  104   257     8     5     5
      4          187.09   3.40    4   15    46     0     0     0
      5          100.97   0.91    2    7     7     0     0     0
      6           14.27   0.24    0    2     7     0     0     1
U2U   3          295.20  10.18    8   36   148     2     2    11
      4        1,006.63  21.22   18  184   485     3    14    40
      5          840.73  10.05   14   83   274     3     7    10
      6          999.73  10.15   20  118   330     4    11    27
U4D   2            7.81   0.64    1    3     4     0     0     0
      3          196.04   9.98    8   67   161     3     4    14
      4           31.68   1.15    1    6    32     0     1     3
      5            6.38   0.16    0    0     9     0     0     1
      6            5.59   0.17    0    3     8     0     0     3
U4F   1          256.84  22.22   32   64   214    11     3     2
U4U   2            6.82   0.34    0    1     4     0     0     0
      3          880.47  51.22   74  391  1137    25    49    65
      4          379.58  14.98   19  142   495     7    15    24
      5           56.36   1.27    4   11    70     1     2     5
      6           47.94   1.09    3   17    46     2     3     4
U6F   1           88.19  26.04    7   47   135     1     1     0
U6U   2            0.76   0.08    1    2     8     0     0     0
      3          186.65  21.30   23  218   502     9    41    60
      4            8.84   0.50    1    7    23     0     1     2
      5            2.05   0.18    0    1     1     0     0     0
      6            0.27   0.02    0    1     1     0     0     0

In [88]:
OwnerShipDesc = {
    0:'Not Assigned',
    1:'State Highway Agency',
    2:'County Highway Agency',
    3:'Town or Township Highway Agency',
    4:'City or Municipal Highway Agency',
    60:'Other Federal Agency',
    62:'Bureau of Indian Affairs',
    64:'U.S. Forest Service'
}
OwnS = [1,4,2,64,60,62,0]
df = pd.DataFrame(Segs_DF_Sel.groupby(['OWNERSHIP'])['Mileage','VMT','K', 'A','B','PedK','PedA','PedB'].aggregate(np.sum))
df.VMT = df.VMT/100000000.0
df = df.loc[OwnS]


df['Ownership Description'] = [OwnerShipDesc[f] for f in OwnS]
df = df[['Ownership Description','Mileage','VMT','K', 'A','B','PedK','PedA','PedB']]
df.loc['Total'] = ['',sum(df.Mileage),sum(df.VMT),sum(df.K),sum(df.A),sum(df.B),sum(df.PedK),sum(df.PedA),sum(df.PedB)]
df[['K', 'A','B','PedK','PedA','PedB']] = df[['K', 'A','B','PedK','PedA','PedB']].astype(int)
df.index.name = None
df.columns.name='Ownership Codes'
pd.options.display.float_format = '{:,.2f}'.format
df

Ownership Codes,Ownership Description,Mileage,VMT,K,A,B,PedK,PedA,PedB
1,State Highway Agency,"29,764.65",409.27,333,532,1663,51,20,13
4,City or Municipal Highway Agency,"3,429.53",86.27,9,33,126,2,6,16
2,County Highway Agency,"1,194.82",9.00,2,3,15,1,0,2
64,U.S. Forest Service,163.31,0.05,0,1,1,0,0,0
60,Other Federal Agency,12.09,0.48,0,0,0,0,0,0
62,Bureau of Indian Affairs,6.34,0.01,0,0,0,0,0,0
0,Not Assigned,367.58,3.87,3,4,7,2,0,1
Total,,"34,938.33",508.95,347,573,1812,56,26,32


In [ ]:
#oc_DF = pd.read_csv(Src_Loc,low_memory=False)
oc_DF['XY'] = ['{}_{}'.format(x,y) for x,y in zip(oc_DF.GIS_UTMX,oc_DF.GIS_UTMY)]
xydf = pd.DataFrame(oc_DF.XY.value_counts())
print(sum(xydf[xydf.XY>4]['XY'])/oc_DF.shape[0])
print(sum(xydf[xydf.XY>4]['XY']),oc_DF.shape[0])
xydf
oc_DF[(oc_DF.GIS_UTMX==344055.5844) & (oc_DF.GIS_UTMY==3883251.81)]

In [11]:
Crash_Dict = {year:hsmpy3.common.FCtoDF(CrashData[year],True) for year in range(2012,2018)}
df = hsmpy3.common.FCtoDF(Routes[2016],True)
Route_Dict = {year:df for year in range(2012,2018)}
Segs_Dict = {year:hsmpy3.common.FCtoDF(Segs[year],True) for year in Years}
Segs_Dict.update({2017:Segs_Dict[2016]})
for year in range(2012,2018):
    Overlay_DF = hsmpy3.common.FCtoDF(Proje)    
    Overlay_DF['SegID_P'] = Overlay_DF.SegID
    Overlay_DF['MP_P'] = Overlay_DF.MP
    for i,r in Overlay_DF.iterrows():
        if r.RID.split('-')[2] == 'M':
            cdf = Crash_Dict[year][Crash_Dict[year]['UCRnum']==r.UCRnum]
            rdf = Route_Dict[year][Route_Dict[year]['ROUTE_ID']==OppositeDirID(r.RID)]
            pnt,disalongr,offset,side = rdf['Shape'].item().queryPointAndDistance(cdf['Shape'].item())
            mp = pnt.firstPoint.M
            sdf = Segs_Dict[year][Segs_Dict[year]['ROUTE_ID']==OppositeDirID(r.RID)]
            if mp == max(list(sdf.END_POINT)):
                sdf = sdf[(sdf.BEG_POINT<=mp) & (sdf.END_POINT>=mp)]
            else:
                sdf = sdf[(sdf.BEG_POINT<=mp) & (sdf.END_POINT>mp)]
            SegID_P = sdf['SegID'].item()
            Overlay_DF.set_value(i,'SegID_P',SegID_P)
            Overlay_DF.set_value(i,'MP_P',mp)
    

In [12]:
Route_Dict = {year:hsmpy3.common.FCtoDF(Routes[year],True) for year in Years}

In [13]:
Segs_Dict = {year:hsmpy3.common.FCtoDF(Segs[year],True) for year in Years}

In [ ]:
# Add Observed Crashes
print(strftime("%Y-%m-%d %H:%M:%S"))

def OppositeDirID(InpID):
    out = InpID[:-2]
    ext = InpID[-1]
    if ext == 'P':
        return('{}-M'.format(out))
    if ext == 'M':
        return('{}-P'.format(out))
for year in Years:
    Tab = hsmpy3.common.CreateOutPath(Project_GDB+'\\Crash_Table',str(year),'')
    arcpy.conversion.TableToTable(
        in_rows=CrashData[year],
        out_name=os.path.basename(Tab),
        out_path=Project_GDB)
    Overlay_Event_Table1 = hsmpy3.common.CreateOutPath(Project_GDB+'\\Crash_Assignment',str(year),'')
    arcpy.OverlayRouteEvents_lr(in_table = Project_GDB+'\\SegAtt_Int'+str(year), 
                                in_event_properties = ' '.join(['RID','LINE','BMP','EMP']), 
                                overlay_table = Tab, 
                                overlay_event_properties = ' '.join(['RID','POINT','MP']), 
                                overlay_type = "INTERSECT", 
                                out_table = Overlay_Event_Table1, 
                                out_event_properties = ' '.join(['RID','POINT','MP']), 
                                zero_length_events = "NO_ZERO", 
                                in_fields = "FIELDS", 
                                build_index="INDEX")
    Overlay_DF = hsmpy3.common.AttributeTabletoDF(Overlay_Event_Table1)    
    Overlay_DF['SegID_P'] = Overlay_DF.SegID
    Overlay_DF['MP_P'] = Overlay_DF.MP
    f = FloatProgress(min=0, max=Overlay_DF.shape[0],description = '{}: MP'.format(str(year)))
    display(f)
    for i,r in Overlay_DF.iterrows():
        if r.RID.split('-')[2] == 'M':
            cdf = Crash_Dict[year][Crash_Dict[year]['UCRnum']==r.UCRnum]
            rdf = Route_Dict[year][Route_Dict[year]['ROUTE_ID']==OppositeDirID(r.RID)]
            pnt,disalongr,offset,side = rdf['Shape'].item().queryPointAndDistance(cdf['Shape'].item())
            mp = pnt.firstPoint.M
            sdf = Segs_Dict[year][Segs_Dict[year]['ROUTE_ID']==OppositeDirID(r.RID)]
            if mp == max(list(sdf.END_POINT)):
                sdf = sdf[(sdf.BEG_POINT<=mp) & (sdf.END_POINT>=mp)]
            else:
                sdf = sdf[(sdf.BEG_POINT<=mp) & (sdf.END_POINT>mp)]
            SegID_P = sdf['SegID'].item()
            Overlay_DF.set_value(i,'SegID_P',SegID_P)
            Overlay_DF.set_value(i,'MP_P',mp)
        f.value += 1

    f = FloatProgress(min=0, max=Overlay_DF.shape[0],description = '{}: Table'.format(str(year)))
    display(f)
    Overlay_DF = Overlay_DF.drop_duplicates(subset='UCRnum')
    arcpy.management.AddField(Overlay_Event_Table1,'SegID_P','Text',100)
    arcpy.management.AddField(Overlay_Event_Table1,'MP_P','Double')
    uc = arcpy.UpdateCursor(Overlay_Event_Table1)
    for row in uc:
        ucr = row.getValue('UCRnum')
        odf = Overlay_DF[Overlay_DF.UCRnum==ucr]
        row.setValue('SegID_P',odf['SegID_P'].item())
        row.setValue('MP_P',odf['MP_P'].item())
        uc.updateRow(row)
        f.value += 1
    del uc
    del row
    
    
    Overlay_DF['K'] = [{True:1,False:0}[v=='K'] for v in Overlay_DF.KABCO]
    Overlay_DF['A'] = [{True:1,False:0}[v=='A'] for v in Overlay_DF.KABCO]
    Overlay_DF['B'] = [{True:1,False:0}[v=='B'] for v in Overlay_DF.KABCO]
    df = pd.DataFrame(Overlay_DF.groupby(['SegID_P'])['K','A','B','PedK','PedA','PedB'].aggregate(np.sum))    
    if 'K' not in [f.name for f in arcpy.ListFields(Segs[year])]:
        arcpy.management.AddField(Segs[year],'K','Short')
    if 'A' not in [f.name for f in arcpy.ListFields(Segs[year])]:
        arcpy.management.AddField(Segs[year],'A','Short')
    if 'B' not in [f.name for f in arcpy.ListFields(Segs[year])]:
        arcpy.management.AddField(Segs[year],'B','Short')
    if 'PedK' not in [f.name for f in arcpy.ListFields(Segs[year])]:
        arcpy.management.AddField(Segs[year],'PedK','Short')
    if 'PedA' not in [f.name for f in arcpy.ListFields(Segs[year])]:
        arcpy.management.AddField(Segs[year],'PedA','Short')
    if 'PedB' not in [f.name for f in arcpy.ListFields(Segs[year])]:
        arcpy.management.AddField(Segs[year],'PedB','Short')
    f = FloatProgress(min=0, max=Segs_Dict[year].shape[0],description = '{}: Segs'.format(str(year)))
    display(f)
    uc = arcpy.UpdateCursor(Segs[year])
    for row in uc:
        segid = row.getValue('SegID')
        if segid in list(df.index):
            r = df.loc[segid]
            row.setValue('K',int(r.K))
            row.setValue('A',int(r.A))
            row.setValue('B',int(r.B))
            row.setValue('PedK',int(r.PedK))
            row.setValue('PedA',int(r.PedA))
            row.setValue('PedB',int(r.PedB))
            uc.updateRow(row)
        f.value += 1
    del uc
    del row

In [78]:
# Add Observed Crashes 2
print(strftime("%Y-%m-%d %H:%M:%S"))

def OppositeDirID(InpID):
    out = InpID[:-2]
    ext = InpID[-1]
    if ext == 'P':
        return('{}-M'.format(out))
    if ext == 'M':
        return('{}-P'.format(out))
for year in [2012,2013,2014,2015,2016,2017]:
    Tab = hsmpy3.common.CreateOutPath(Project_GDB+'\\Crash_Table',str(year),'')
    arcpy.conversion.TableToTable(
        in_rows=CrashData[year],
        out_name=os.path.basename(Tab),
        out_path=Project_GDB)
    Overlay_Event_Table1 = hsmpy3.common.CreateOutPath(Project_GDB+'\\Crash_Assignment',str(year),'')
    arcpy.OverlayRouteEvents_lr(in_table = Project_GDB+'\\SegInt_Att_'+str(year), 
                                in_event_properties = ' '.join(['ROUTE_ID','LINE','BEG_POINT','END_POINT']), 
                                overlay_table = Tab, 
                                overlay_event_properties = ' '.join(['RID','POINT','MP']), 
                                overlay_type = "INTERSECT", 
                                out_table = Overlay_Event_Table1, 
                                out_event_properties = ' '.join(['RID','POINT','MP']), 
                                zero_length_events = "NO_ZERO", 
                                in_fields = "FIELDS", 
                                build_index="INDEX")
    Overlay_DF = hsmpy3.common.FCtoDF(Overlay_Event_Table1)    
    Overlay_DF['SegID_P'] = Overlay_DF.SegID
    Overlay_DF['MP_P'] = Overlay_DF.MP
    f = FloatProgress(min=0, max=Overlay_DF.shape[0],description = '{}: MP'.format(str(year)))
    display(f)
    for i,r in Overlay_DF.iterrows():
        if r.RID.split('-')[2] == 'M':
            cdf = Crash_Dict[year][Crash_Dict[year]['UCRnum']==r.UCRnum]
            rdf = Route_Dict[year][Route_Dict[year]['ROUTE_ID']==OppositeDirID(r.RID)]
            pl =arcpy.Polyline(arcpy.Array([s[0] for s in list(rdf.Shape)]),NAD83_NM,True,True)
            pnt,disalongr,offset,side = pl.queryPointAndDistance(cdf['Shape'].item())
            mp = pnt.firstPoint.M
            sdf = Segs_Dict[year][Segs_Dict[year]['ROUTE_ID']==OppositeDirID(r.RID)]
            if mp == max(list(sdf.END_POINT)):
                sdf = sdf[(sdf.BEG_POINT<=mp) & (sdf.END_POINT>=mp)]
            else:
                sdf = sdf[(sdf.BEG_POINT<=mp) & (sdf.END_POINT>mp)]
            SegID_P = sdf['SegID'].item()
            Overlay_DF.set_value(i,'SegID_P',SegID_P)
            Overlay_DF.set_value(i,'MP_P',mp)
        f.value += 1

    f = FloatProgress(min=0, max=Overlay_DF.shape[0],description = '{}: Table'.format(str(year)))
    display(f)
    Overlay_DF = Overlay_DF.drop_duplicates(subset='UCRnum')
    arcpy.management.AddField(Overlay_Event_Table1,'SegID_P','Text',100)
    arcpy.management.AddField(Overlay_Event_Table1,'MP_P','Double')
    uc = arcpy.UpdateCursor(Overlay_Event_Table1)
    for row in uc:
        ucr = row.getValue('UCRnum')
        odf = Overlay_DF[Overlay_DF.UCRnum==ucr]
        row.setValue('SegID_P',odf['SegID_P'].item())
        row.setValue('MP_P',odf['MP_P'].item())
        uc.updateRow(row)
        f.value += 1
    del uc
    del row
    
    
    Overlay_DF['K'] = [{True:1,False:0}[v=='K'] for v in Overlay_DF.KABCO]
    Overlay_DF['A'] = [{True:1,False:0}[v=='A'] for v in Overlay_DF.KABCO]
    Overlay_DF['B'] = [{True:1,False:0}[v=='B'] for v in Overlay_DF.KABCO]
    
   
    df = pd.DataFrame(Overlay_DF[Overlay_DF.IntProx==0].groupby(['SegID_P'])['K','A','B','PedK','PedA','PedB'].aggregate(np.sum))    
    if 'K' not in [f.name for f in arcpy.ListFields(Segs[year])]:
        arcpy.management.AddField(Segs[year],'K','Short')
    if 'A' not in [f.name for f in arcpy.ListFields(Segs[year])]:
        arcpy.management.AddField(Segs[year],'A','Short')
    if 'B' not in [f.name for f in arcpy.ListFields(Segs[year])]:
        arcpy.management.AddField(Segs[year],'B','Short')
    if 'PedK' not in [f.name for f in arcpy.ListFields(Segs[year])]:
        arcpy.management.AddField(Segs[year],'PedK','Short')
    if 'PedA' not in [f.name for f in arcpy.ListFields(Segs[year])]:
        arcpy.management.AddField(Segs[year],'PedA','Short')
    if 'PedB' not in [f.name for f in arcpy.ListFields(Segs[year])]:
        arcpy.management.AddField(Segs[year],'PedB','Short')
    f = FloatProgress(min=0, max=Segs_Dict[year].shape[0],description = '{}: Segs'.format(str(year)))
    display(f)
    uc = arcpy.UpdateCursor(Segs[year])
    for row in uc:
        segid = row.getValue('SegID')
        if segid in list(df.index):
            r = df.loc[segid]
            row.setValue('K',int(r.K))
            row.setValue('A',int(r.A))
            row.setValue('B',int(r.B))
            row.setValue('PedK',int(r.PedK))
            row.setValue('PedA',int(r.PedA))
            row.setValue('PedB',int(r.PedB))
            uc.updateRow(row)
        f.value += 1
        
        
    df = pd.DataFrame(Overlay_DF[Overlay_DF.IntProx>0].groupby(['Int_ID'])['K','A','B','PedK','PedA','PedB'].aggregate(np.sum))   
    Int_FC = Project_GDB + '\\Int_' + str(year)
    if 'K' not in [f.name for f in arcpy.ListFields(Int_FC)]:
        arcpy.management.AddField(Int_FC,'K','Short')
    if 'A' not in [f.name for f in arcpy.ListFields(Int_FC)]:
        arcpy.management.AddField(Int_FC,'A','Short')
    if 'B' not in [f.name for f in arcpy.ListFields(Int_FC)]:
        arcpy.management.AddField(Int_FC,'B','Short')
    if 'PedK' not in [f.name for f in arcpy.ListFields(Int_FC)]:
        arcpy.management.AddField(Int_FC,'PedK','Short')
    if 'PedA' not in [f.name for f in arcpy.ListFields(Int_FC)]:
        arcpy.management.AddField(Int_FC,'PedA','Short')
    if 'PedB' not in [f.name for f in arcpy.ListFields(Int_FC)]:
        arcpy.management.AddField(Int_FC,'PedB','Short')
    f = FloatProgress(min=0, max=Int_Dict[year].shape[0],description = '{}: Ints'.format(str(year)))
    display(f)
    uc = arcpy.UpdateCursor(Int_FC)
    for row in uc:
        segid = row.getValue('Int_ID')
        if segid in list(df.index):
            r = df.loc[segid]
            row.setValue('K',int(r.K))
            row.setValue('A',int(r.A))
            row.setValue('B',int(r.B))
            row.setValue('PedK',int(r.PedK))
            row.setValue('PedA',int(r.PedA))
            row.setValue('PedB',int(r.PedB))
            uc.updateRow(row)
        f.value += 1
        
    if 'SegID' not in [f.name for f in arcpy.ListFields(CrashData[year])]:
        arcpy.management.AddField(CrashData[year],'SegID','Text',100)
    if 'Int_ID' not in [f.name for f in arcpy.ListFields(CrashData[year])]:
        arcpy.management.AddField(CrashData[year],'Int_ID','Long')
    if 'FType' not in [f.name for f in arcpy.ListFields(CrashData[year])]:
        arcpy.management.AddField(CrashData[year],'FType','Text',100)
    f = FloatProgress(min=0, max=Overlay_DF.shape[0],description = '{}: Crashes'.format(str(year)))
    display(f)
    uc = arcpy.UpdateCursor(CrashData[year])
    for row in uc:
        ucr = row.getValue('UCRnum')
        if ucr in list(Overlay_DF.UCRnum):
            r = Overlay_DF[Overlay_DF.UCRnum==ucr].iloc[0]
            if r.IntProx==0:
                row.setValue('SegID',r.SegID)
                row.setValue('FType',r.FType)
            else:
                row.setValue('Int_ID',int(r.Int_ID))
                row.setValue('FType',r.IFType)
            uc.updateRow(row)
        f.value += 1
    del uc
    del row

2018-08-06 17:28:40


FloatProgress(value=0.0, description='2012: MP', max=3471.0)

FloatProgress(value=0.0, description='2012: Table', max=3471.0)

FloatProgress(value=0.0, description='2012: Segs', max=9116.0)

FloatProgress(value=0.0, description='2012: Ints', max=103874.0)

...

FloatProgress(value=0.0, description='2016: Table', max=5223.0)

FloatProgress(value=0.0, description='2016: Segs', max=224743.0)

FloatProgress(value=0.0, description='2016: Ints', max=103874.0)

FloatProgress(value=0.0, description='2016: Crashes', max=5038.0)

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000732: Input Event Table: Dataset \\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO00_OtherStates\NewMexico\3_WorkData\7_NetworkScreening\5.NM_NS_GDB.gdb\SegInt_Att_2017 does not exist or is not supported
Failed to execute (OverlayRouteEvents).


In [4]:
Diss_Fields = ['F_SYSTEM','OWNERSHIP','URBAN_CODE','SegID','FType','Int_ID','IFType']
for year in range(2012,2017):
    Tab = hsmpy3.common.CreateOutPath(Project_GDB+'\\Crash_Table',str(year),'')
    arcpy.conversion.TableToTable(
        in_rows=CrashData[year],
        out_name=os.path.basename(Tab),
        out_path=Project_GDB)
    for f in [F.name for F in arcpy.ListFields(Tab)]:
        if f in Diss_Fields:
            arcpy.management.DeleteField(Tab,f)
            
    Diss = hsmpy3.common.CreateOutPath(MainFile=Project_GDB+'\\SegAtt_Int_Diss',Extension='',appendix=str(year))
    arcpy.DissolveRouteEvents_lr (in_events = Project_GDB+'\\SegAtt_Int_'+str(year), 
                                  in_event_properties = ' '.join(['RID','LINE','BMP','EMP']), 
                                  dissolve_field = Diss_Fields, 
                                  out_table = Diss, 
                                  out_event_properties = ' '.join(['RID','LINE','BMP','EMP']), 
                                  dissolve_type = 'DISSOLVE', 
                                  build_index = 'INDEX')  

    Overlay_Event_Table1 = hsmpy3.common.CreateOutPath(Project_GDB+'\\Crash_Assignment',str(year),'')
    arcpy.OverlayRouteEvents_lr(in_table = Tab, 
                                in_event_properties = ' '.join(['RID','POINT','MP']),
                                overlay_table = Diss, 
                                overlay_event_properties = ' '.join(['RID','LINE','BMP','EMP']),  
                                overlay_type = "INTERSECT", 
                                out_table = Overlay_Event_Table1, 
                                out_event_properties = ' '.join(['RID','POINT','MP']), 
                                zero_length_events = "NO_ZERO", 
                                in_fields = "FIELDS", 
                                build_index="INDEX")
    print(year)

2012
2013
2014
2015
2016


In [5]:
Diss_Fields = ['F_SYSTEM','OWNERSHIP','URBAN_CODE','SegID','FType','Int_ID','IFType']
if 1<2:

    Tab = hsmpy3.common.CreateOutPath(Project_GDB+'\\Crash_Table',str(2017),'')
    arcpy.conversion.TableToTable(
        in_rows=CrashData[2017],
        out_name=os.path.basename(Tab),
        out_path=Project_GDB)
    for f in [F.name for F in arcpy.ListFields(Tab)]:
        if f in Diss_Fields:
            arcpy.management.DeleteField(Tab,f)
            
    Diss = hsmpy3.common.CreateOutPath(MainFile=Project_GDB+'\\SegAtt_Int_Diss',Extension='',appendix=str(2016))
    arcpy.DissolveRouteEvents_lr (in_events = Project_GDB+'\\SegAtt_Int_'+str(2016), 
                                  in_event_properties = ' '.join(['RID','LINE','BMP','EMP']), 
                                  dissolve_field = Diss_Fields, 
                                  out_table = Diss, 
                                  out_event_properties = ' '.join(['RID','LINE','BMP','EMP']), 
                                  dissolve_type = 'DISSOLVE', 
                                  build_index = 'INDEX')  

    Overlay_Event_Table1 = hsmpy3.common.CreateOutPath(Project_GDB+'\\Crash_Assignment',str(2017),'')
    arcpy.OverlayRouteEvents_lr(in_table = Tab, 
                                in_event_properties = ' '.join(['RID','POINT','MP']),
                                overlay_table = Diss, 
                                overlay_event_properties = ' '.join(['RID','LINE','BMP','EMP']),  
                                overlay_type = "INTERSECT", 
                                out_table = Overlay_Event_Table1, 
                                out_event_properties = ' '.join(['RID','POINT','MP']), 
                                zero_length_events = "NO_ZERO", 
                                in_fields = "FIELDS", 
                                build_index="INDEX")
    print(2017)

2017


In [38]:
for year in Years:
    df = hsmpy3.common.FCtoDF(Project_GDB+'\\Crash_Assignment_'+str(year))
    if not 'F_SYSTEM' in [f.name for f in arcpy.ListFields(CrashData[year])]:
        arcpy.management.AddField(CrashData[year],'F_SYSTEM','Single')
    if not 'FACILITY_TYPE' in [f.name for f in arcpy.ListFields(CrashData[year])]:
        arcpy.management.AddField(CrashData[year],'FACILITY_TYPE','Single')
    if not 'URBAN_CODE' in [f.name for f in arcpy.ListFields(CrashData[year])]:
        arcpy.management.AddField(CrashData[year],'URBAN_CODE','Single')
    if not 'OWNERSHIP' in [f.name for f in arcpy.ListFields(CrashData[year])]:
        arcpy.management.AddField(CrashData[year],'OWNERSHIP','Single')
    if not 'FType' in [f.name for f in arcpy.ListFields(CrashData[year])]:
        arcpy.management.AddField(CrashData[year],'FType','Text')
    if not 'IFType' in [f.name for f in arcpy.ListFields(CrashData[year])]:
        arcpy.management.AddField(CrashData[year],'IFType','Text')
    if not 'Int_ID' in [f.name for f in arcpy.ListFields(CrashData[year])]:
        arcpy.management.AddField(CrashData[year],'Int_ID','Long')
    if not 'SegID' in [f.name for f in arcpy.ListFields(CrashData[year])]:
        arcpy.management.AddField(CrashData[year],'SegID','Text',50)
    uc = arcpy.UpdateCursor(CrashData[year])
    for r in uc:
        ucr = r.getValue('UCRnum')
        cdf = df[df['UCRnum'] == ucr]
        if cdf.shape[0]==1:
            if cdf['F_SYSTEM'].item() !='':
                r.setValue('F_SYSTEM'     ,int(cdf['F_SYSTEM'].item()))
            if cdf['FACILITY_TYPE'].item() !='':
                r.setValue('FACILITY_TYPE',int(cdf['FACILITY_TYPE'].item()))
            if cdf['URBAN_CODE'].item() !='':
                r.setValue('URBAN_CODE',int(cdf['URBAN_CODE'].item()))
            if cdf['OWNERSHIP'].item() !='':
                r.setValue('OWNERSHIP',int(cdf['OWNERSHIP'].item()))
            if cdf['FType'].item() !='':
                r.setValue('FType',cdf['FType'].item())
            if cdf['IFType'].item() !='':
                r.setValue('IFType',cdf['IFType'].item())
            if cdf['Int_ID'].item() !='':
                r.setValue('Int_ID',cdf['Int_ID'].item())
            if cdf['SegID'].item() !='':
                r.setValue('SegID',cdf['SegID'].item())
            uc.updateRow(r)
        del uc
        del r
    print(year)

2012
2013
2014
2015
2016


In [47]:
# Adds IFType from Int_XXXX to SegInt_Att_XXXX
Int_Dict = {year:hsmpy3.common.FCtoDF(Project_GDB + '\\Int_' + str(year)) for year in Years}
for year in Years:
    fc = Project_GDB+'\\SegInt_Att_'+str(year)
    if not 'IFType' in [f.name for f in arcpy.ListFields(fc)]:
        arcpy.management.AddField(fc,'IFType','Text',10)
    uc = arcpy.UpdateCursor(Project_GDB+'\\SegInt_Att_'+str(year))
    df = Int_Dict[year]
    df.index = list(df.Int_ID)
    df = df[['IFType']]
    df = df[~pd.isnull(df.IFType)]
    Dict = df.to_dict('index')
    for row in uc:
        int_id = row.getValue('Int_ID')
        if int_id in Dict.keys():
            row.setValue('IFType',Dict[int_id]['IFType'])
            uc.updateRow(row)
    del uc
    del row
    print(year)

2012
2013
2014
2015
2016


In [103]:
Int_Dict = {year:hsmpy3.common.FCtoDF(Project_GDB + '\\Int_' + str(year)) for year in Years}

In [105]:
IntDesc = {
    'R3ST':'Rural Three-Leg Stop Control',
    'R4ST':'Rural Four-Leg Stop Control',
    'R4SG':'Rural Four-Leg Signal Control',
    'RM3ST':'Rural MultiLane Three-Leg Stop Control',
    'RM4ST':'Rural MultiLane Four-Leg Stop Control',
    'RM4SG':'Rural MultiLane Four-Leg Signal Control',
    'U3ST':'Urban Three-Leg Stop Control',
    'U4ST':'Urban Four-Leg Stop Control',
    'U3SG':'Urban Three-Leg Signal Control',
    'U4SG':'Urban Four-Leg Signal Control',
    'Total':''
}
Sum = pd.DataFrame()
for year in Years:
    df = Int_Dict[year]
    rdf = pd.DataFrame(df.IFType.value_counts()).T
    rdf.index = [year]
    Sum = pd.concat([Sum,rdf])
Sum = Sum[['R3ST','R4ST','R4SG','RM3ST','RM4ST','RM4SG','U3ST','U4ST','U3SG','U4SG']]
Sum = Sum.T
Sum.loc['Total'] = [sum(Sum[c]) for c in list(Sum)]
Sum['Type Description'] = [IntDesc[v] for v in list(Sum.index)]
Sum = Sum[['Type Description',2012,2013,2014,2015,2016]]
Sum.columns.name = 'Type Code'
Sum

Type Code,Type Description,2012,2013,2014,2015,2016
R3ST,Rural Three-Leg Stop Control,5539,5240,5715,6719,8656
R4ST,Rural Four-Leg Stop Control,1203,1133,1230,1389,1737
R4SG,Rural Four-Leg Signal Control,24,24,23,21,17
RM3ST,Rural MultiLane Three-Leg Stop Control,1181,1192,1213,1237,1191
RM4ST,Rural MultiLane Four-Leg Stop Control,313,324,337,324,354
RM4SG,Rural MultiLane Four-Leg Signal Control,34,28,27,19,18
U3ST,Urban Three-Leg Stop Control,6298,7690,8113,10326,10248
U4ST,Urban Four-Leg Stop Control,2811,3526,3794,4396,4380
U3SG,Urban Three-Leg Signal Control,312,332,340,345,339
U4SG,Urban Four-Leg Signal Control,695,748,784,794,797


In [106]:
Sum = pd.DataFrame()
for year in Years:
    df = Int_Dict[year]
    df = df[(df.AADT_Major>0) & (df.AADT_Minor>0)]
    rdf = pd.DataFrame(df.IFType.value_counts()).T
    rdf.index = [year]
    Sum = pd.concat([Sum,rdf])
Sum = Sum[['R3ST','R4ST','R4SG','RM3ST','RM4ST','RM4SG','U3ST','U4ST','U3SG','U4SG']]
Sum = Sum.T
Sum.loc['Total'] = [sum(Sum[c]) for c in list(Sum)]
Sum['Type Description'] = [IntDesc[v] for v in list(Sum.index)]
Sum = Sum[['Type Description',2012,2013,2014,2015,2016]]
Sum.columns.name = 'Type Code'
Sum

Type Code,Type Description,2012,2013,2014,2015,2016
R3ST,Rural Three-Leg Stop Control,314,171,201,315,667
R4ST,Rural Four-Leg Stop Control,177,112,124,186,339
R4SG,Rural Four-Leg Signal Control,13,17,16,17,13
RM3ST,Rural MultiLane Three-Leg Stop Control,123,73,77,149,190
RM4ST,Rural MultiLane Four-Leg Stop Control,65,50,55,58,77
RM4SG,Rural MultiLane Four-Leg Signal Control,26,20,21,16,18
U3ST,Urban Three-Leg Stop Control,416,440,490,819,839
U4ST,Urban Four-Leg Stop Control,420,448,508,704,715
U3SG,Urban Three-Leg Signal Control,143,157,171,210,208
U4SG,Urban Four-Leg Signal Control,394,503,582,641,639


In [126]:
FTypes = ['R3ST','R4ST','R4SG','RM3ST','RM4ST','RM4SG','U3ST','U4ST','U3SG','U4SG']
IntDesc = {
    'R3ST':'Rural Three-Leg Stop Control',
    'R4ST':'Rural Four-Leg Stop Control',
    'R4SG':'Rural Four-Leg Signal Control',
    'RM3ST':'Rural MultiLane Three-Leg Stop Control',
    'RM4ST':'Rural MultiLane Four-Leg Stop Control',
    'RM4SG':'Rural MultiLane Four-Leg Signal Control',
    'U3ST':'Urban Three-Leg Stop Control',
    'U4ST':'Urban Four-Leg Stop Control',
    'U3SG':'Urban Three-Leg Signal Control',
    'U4SG':'Urban Four-Leg Signal Control',
}
Int_DF['Count'] = 1
Int_DF_Sel = Int_DF[(Int_DF.IFType.isin(FTypes)) & (Int_DF.AADT_Major>0) & (Int_DF.AADT_Minor>0)]
df = pd.DataFrame(Int_DF_Sel.groupby(['IFType'])['Count','K','A','B','PedK','PedA','PedB'].aggregate(np.sum))
df = df.loc[FTypes]
df['Peer Group Description'] = [IntDesc[f] for f in FTypes]
df = df[['Peer Group Description','Count','K','A','B','PedK','PedA','PedB']]
df.loc['Total'] = ['',sum(df.Count),sum(df.K),sum(df.A),sum(df.B),sum(df.PedK),sum(df.PedA),sum(df.PedB)]
df[['K', 'A','B','PedK','PedA','PedB']] = df[['K', 'A','B','PedK','PedA','PedB']].astype(int)
df.index.name = None
df.columns.name='Peer Group Codes'
pd.options.display.float_format = '{:,.2f}'.format
df

Peer Group Codes,Peer Group Description,Count,K,A,B,PedK,PedA,PedB
R3ST,Rural Three-Leg Stop Control,1668,18,27,88,0,1,2
R4ST,Rural Four-Leg Stop Control,938,8,16,65,0,0,3
R4SG,Rural Four-Leg Signal Control,76,2,3,25,1,0,1
RM3ST,Rural MultiLane Three-Leg Stop Control,612,7,22,66,2,0,3
RM4ST,Rural MultiLane Four-Leg Stop Control,305,8,12,43,0,1,1
RM4SG,Rural MultiLane Four-Leg Signal Control,101,3,15,72,1,2,11
U3ST,Urban Three-Leg Stop Control,3004,15,81,292,4,5,26
U4ST,Urban Four-Leg Stop Control,2795,10,89,290,2,6,8
U3SG,Urban Three-Leg Signal Control,889,12,157,390,4,10,20
U4SG,Urban Four-Leg Signal Control,2759,68,620,1774,21,60,107


In [78]:
# Add Observed Crashes 2
print(strftime("%Y-%m-%d %H:%M:%S"))

def OppositeDirID(InpID):
    out = InpID[:-2]
    ext = InpID[-1]
    if ext == 'P':
        return('{}-M'.format(out))
    if ext == 'M':
        return('{}-P'.format(out))
for year in [2012,2013,2014,2015,2016,2017]:
    Overlay_DF = hsmpy3.common.FCtoDF(Overlay_Event_Table1)    
    Overlay_DF['SegID_P'] = Overlay_DF.SegID
    Overlay_DF['MP_P'] = Overlay_DF.MP
    f = FloatProgress(min=0, max=Overlay_DF.shape[0],description = '{}: MP'.format(str(year)))
    display(f)
    for i,r in Overlay_DF.iterrows():
        if r.RID.split('-')[2] == 'M':
            cdf = Crash_Dict[year][Crash_Dict[year]['UCRnum']==r.UCRnum]
            rdf = Route_Dict[year][Route_Dict[year]['ROUTE_ID']==OppositeDirID(r.RID)]
            pl =arcpy.Polyline(arcpy.Array([s[0] for s in list(rdf.Shape)]),NAD83_NM,True,True)
            pnt,disalongr,offset,side = pl.queryPointAndDistance(cdf['Shape'].item())
            mp = pnt.firstPoint.M
            sdf = Segs_Dict[year][Segs_Dict[year]['ROUTE_ID']==OppositeDirID(r.RID)]
            if mp == max(list(sdf.END_POINT)):
                sdf = sdf[(sdf.BEG_POINT<=mp) & (sdf.END_POINT>=mp)]
            else:
                sdf = sdf[(sdf.BEG_POINT<=mp) & (sdf.END_POINT>mp)]
            SegID_P = sdf['SegID'].item()
            Overlay_DF.set_value(i,'SegID_P',SegID_P)
            Overlay_DF.set_value(i,'MP_P',mp)
        f.value += 1

    f = FloatProgress(min=0, max=Overlay_DF.shape[0],description = '{}: Table'.format(str(year)))
    display(f)
    Overlay_DF = Overlay_DF.drop_duplicates(subset='UCRnum')
    arcpy.management.AddField(Overlay_Event_Table1,'SegID_P','Text',100)
    arcpy.management.AddField(Overlay_Event_Table1,'MP_P','Double')
    uc = arcpy.UpdateCursor(Overlay_Event_Table1)
    for row in uc:
        ucr = row.getValue('UCRnum')
        odf = Overlay_DF[Overlay_DF.UCRnum==ucr]
        row.setValue('SegID_P',odf['SegID_P'].item())
        row.setValue('MP_P',odf['MP_P'].item())
        uc.updateRow(row)
        f.value += 1
    del uc
    del row
    
    
    Overlay_DF['K'] = [{True:1,False:0}[v=='K'] for v in Overlay_DF.KABCO]
    Overlay_DF['A'] = [{True:1,False:0}[v=='A'] for v in Overlay_DF.KABCO]
    Overlay_DF['B'] = [{True:1,False:0}[v=='B'] for v in Overlay_DF.KABCO]
    
   
    df = pd.DataFrame(Overlay_DF[Overlay_DF.IntProx==0].groupby(['SegID_P'])['K','A','B','PedK','PedA','PedB'].aggregate(np.sum))    
    if 'K' not in [f.name for f in arcpy.ListFields(Segs[year])]:
        arcpy.management.AddField(Segs[year],'K','Short')
    if 'A' not in [f.name for f in arcpy.ListFields(Segs[year])]:
        arcpy.management.AddField(Segs[year],'A','Short')
    if 'B' not in [f.name for f in arcpy.ListFields(Segs[year])]:
        arcpy.management.AddField(Segs[year],'B','Short')
    if 'PedK' not in [f.name for f in arcpy.ListFields(Segs[year])]:
        arcpy.management.AddField(Segs[year],'PedK','Short')
    if 'PedA' not in [f.name for f in arcpy.ListFields(Segs[year])]:
        arcpy.management.AddField(Segs[year],'PedA','Short')
    if 'PedB' not in [f.name for f in arcpy.ListFields(Segs[year])]:
        arcpy.management.AddField(Segs[year],'PedB','Short')
    f = FloatProgress(min=0, max=Segs_Dict[year].shape[0],description = '{}: Segs'.format(str(year)))
    display(f)
    uc = arcpy.UpdateCursor(Segs[year])
    for row in uc:
        segid = row.getValue('SegID')
        if segid in list(df.index):
            r = df.loc[segid]
            row.setValue('K',int(r.K))
            row.setValue('A',int(r.A))
            row.setValue('B',int(r.B))
            row.setValue('PedK',int(r.PedK))
            row.setValue('PedA',int(r.PedA))
            row.setValue('PedB',int(r.PedB))
            uc.updateRow(row)
        f.value += 1
        
        
    df = pd.DataFrame(Overlay_DF[Overlay_DF.IntProx>0].groupby(['Int_ID'])['K','A','B','PedK','PedA','PedB'].aggregate(np.sum))   
    Int_FC = Project_GDB + '\\Int_' + str(year)
    if 'K' not in [f.name for f in arcpy.ListFields(Int_FC)]:
        arcpy.management.AddField(Int_FC,'K','Short')
    if 'A' not in [f.name for f in arcpy.ListFields(Int_FC)]:
        arcpy.management.AddField(Int_FC,'A','Short')
    if 'B' not in [f.name for f in arcpy.ListFields(Int_FC)]:
        arcpy.management.AddField(Int_FC,'B','Short')
    if 'PedK' not in [f.name for f in arcpy.ListFields(Int_FC)]:
        arcpy.management.AddField(Int_FC,'PedK','Short')
    if 'PedA' not in [f.name for f in arcpy.ListFields(Int_FC)]:
        arcpy.management.AddField(Int_FC,'PedA','Short')
    if 'PedB' not in [f.name for f in arcpy.ListFields(Int_FC)]:
        arcpy.management.AddField(Int_FC,'PedB','Short')
    f = FloatProgress(min=0, max=Int_Dict[year].shape[0],description = '{}: Ints'.format(str(year)))
    display(f)
    uc = arcpy.UpdateCursor(Int_FC)
    for row in uc:
        segid = row.getValue('Int_ID')
        if segid in list(df.index):
            r = df.loc[segid]
            row.setValue('K',int(r.K))
            row.setValue('A',int(r.A))
            row.setValue('B',int(r.B))
            row.setValue('PedK',int(r.PedK))
            row.setValue('PedA',int(r.PedA))
            row.setValue('PedB',int(r.PedB))
            uc.updateRow(row)
        f.value += 1
        
    if 'SegID' not in [f.name for f in arcpy.ListFields(CrashData[year])]:
        arcpy.management.AddField(CrashData[year],'SegID','Text',100)
    if 'Int_ID' not in [f.name for f in arcpy.ListFields(CrashData[year])]:
        arcpy.management.AddField(CrashData[year],'Int_ID','Long')
    if 'FType' not in [f.name for f in arcpy.ListFields(CrashData[year])]:
        arcpy.management.AddField(CrashData[year],'FType','Text',100)
    f = FloatProgress(min=0, max=Overlay_DF.shape[0],description = '{}: Crashes'.format(str(year)))
    display(f)
    uc = arcpy.UpdateCursor(CrashData[year])
    for row in uc:
        ucr = row.getValue('UCRnum')
        if ucr in list(Overlay_DF.UCRnum):
            r = Overlay_DF[Overlay_DF.UCRnum==ucr].iloc[0]
            if r.IntProx==0:
                row.setValue('SegID',r.SegID)
                row.setValue('FType',r.FType)
            else:
                row.setValue('Int_ID',int(r.Int_ID))
                row.setValue('FType',r.IFType)
            uc.updateRow(row)
        f.value += 1
    del uc
    del row

2018-08-06 17:28:40


FloatProgress(value=0.0, description='2012: MP', max=3471.0)

FloatProgress(value=0.0, description='2012: Table', max=3471.0)

FloatProgress(value=0.0, description='2012: Segs', max=9116.0)

FloatProgress(value=0.0, description='2012: Ints', max=103874.0)

...

FloatProgress(value=0.0, description='2016: Table', max=5223.0)

FloatProgress(value=0.0, description='2016: Segs', max=224743.0)

FloatProgress(value=0.0, description='2016: Ints', max=103874.0)

FloatProgress(value=0.0, description='2016: Crashes', max=5038.0)

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000732: Input Event Table: Dataset \\CHCFPP01\Proj\ILDOT\650511SAFETYPROGRAM\4_WorkData\WO00_OtherStates\NewMexico\3_WorkData\7_NetworkScreening\5.NM_NS_GDB.gdb\SegInt_Att_2017 does not exist or is not supported
Failed to execute (OverlayRouteEvents).


In [35]:
#Overlay_DF = hsmpy3.common.FCtoDF(Project_GDB+'\\Crash_Assignment_2016',selectedFields=['SegID','Int_ID','FType','IFType','KABCO'])   
Assign_DF = pd.DataFrame()
for year in Years:
    df = hsmpy3.common.FCtoDF(Project_GDB+'\\Crash_Assignment_'+str(year))   
    df['Year'] = year
    Assign_DF = pd.concat([Assign_DF,df])
Assign_DF = Assign_DF.fillna(0)
Assign_DF.shape

(22649, 23)

In [36]:
Assign_DF.index = range(len(Assign_DF))
sel_i = Assign_DF.index[Assign_DF.FType==0]
Assign_DF.loc[sel_i,'FType'] = 'Other_Seg'
sel_i = Assign_DF.index[Assign_DF.IFType==0]
Assign_DF.loc[sel_i,'IFType'] = 'Other_Int'
Assign_DF['CFType'] = list(Assign_DF.FType)
sel_i = Assign_DF.index[[(len(str(f))>1) for f in Assign_DF.IFType]]
Assign_DF.loc[sel_i,['CFType']] = Assign_DF.loc[sel_i,'IFType']
df = pd.DataFrame(Assign_DF.CFType.value_counts())
df
#Assign_DF.groupby(['FType'])['Killed','ClassA','ClassB'].aggregate(sum)

,CFType
Other_Int,4985
U4SG,2819
U3ST,2741
Other_Seg,2122
R2U,1773
U4ST,1542
R4F,871
U3SG,844
R3ST,818
R2L,508
